# FastScan AWS SageMaker Pipeline Implementation

The implementation in this jupyter notebook can only be used in AWS SageMaker Studio Code Editor.

https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-pipelines/tabular/abalone_build_train_deploy/sagemaker-pipelines-preprocess-train-evaluate-batch-transform.html#Define-a-Processing-Step-for-Feature-Engineering

# 1. Install required dependencies

In [1]:
%pip install sagemaker-training
%pip install opencv-python
%pip install numpy
%pip install sagemaker --upgrade
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1.1 Install required packages for OpenCV

In [2]:
!sudo apt-get update -y
!sudo apt-get install ffmpeg libsm6 libxext6 -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease 
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3664 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2639 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1531 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2941 kB]
Fetched 11.2 MB in 1s (8583 kB/s)                       
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1build2).
libxext6 is already the newest version (2:1.3.4-1build1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 n

## 1.2 Import Packages

In [1]:
import cv2
import os
import numpy
import sagemaker
import sys




/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# 2. Create SageMaker Pipeline Session

In [12]:
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()

region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

pipeline_session = PipelineSession()

default_bucket = sagemaker_session.default_bucket()

model_package_group_name = f"FastScanModelPackageGroupName"

# 3. Define Parameters in the Pipeline for Pipeline Execution

In [13]:
image_input_data_uri: str = "s3://angkokleong-bucket/datasets/raw_custom_image_dataset/"
label_input_data_uri: str = "s3://angkokleong-bucket/label-data/"

In [22]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

#There can be multiple input data


processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")
input_image_data = ParameterString(
    name="InputImageData",
    default_value=image_input_data_uri
)

input_label_data = ParameterString(
    name="InputLabelData",
    default_value=label_input_data_uri
)

mAP50_threshold = ParameterFloat(name="mAP50Threshold", default_value=0.9)
mAP50to95_threshold = ParameterFloat(name="mAP50to95threshold", default_value=0.8)

# 4. Prepare Train, Val and Test dataset and Label Data

Image file split to create train, test and val dataset for YOLO model training

In [23]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn


sklearn_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version="1.2-1",
    role=role,
    py_version="py3",
    instance_count=processing_instance_count,
    instance_type="ml.m5.xlarge",
    sagemaker_session=pipeline_session,
    base_job_name="image_and_label_data_train_test_split_process"
)


In [24]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
import project_library.file_manager

# Project Directory in this project
project_library_folder_path: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/home/sagemaker-user/user-default-efs/FastScan/project_library"))


# Docker context
ROOT_INPUT_FOLDER_PATH: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/opt/ml/processing/input"))
ROOT_OUTPUT_FOLDER_PATH: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/opt/ml/processing/output"))

aws_dataset_folder_path: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/opt/ml/processing/input/aws_datasets"))
fastscan_dataset_folder_path: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/opt/ml/processing/input/aws_datasets/fastscandataset"))
raw_image_dataset_folder_path: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/opt/ml/processing/input/aws_datasets/fastscandataset/raw_custom_image_dataset"))
raw_label_dataset_folder_path: str = str(project_library.file_manager.FileInformation.get_absolute_folder_location("/opt/ml/processing/input/aws_datasets/fastscandataset/raw_custom_label_dataset"))



processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_image_data, destination=raw_image_dataset_folder_path),
        ProcessingInput(source=input_label_data, destination=raw_label_dataset_folder_path)
    ],
    dependencies=[str("/home/sagemaker-user/user-default-efs/FastScan/project_library"), str("/home/sagemaker-user/user-default-efs/FastScan/aws_sagemaker_pipeline/preprocessing/requirements.txt")],
    code=str("/home/sagemaker-user/user-default-efs/FastScan/aws_sagemaker_pipeline/preprocessing/image_preprocessing_script.py"),
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/input/aws_datasets/fastscandataset/raw/train", destination="s3://angkokleong-bucket/datasets/fastscandataset/images/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/input/aws_datasets/fastscandataset/raw/test", destination="s3://angkokleong-bucket/datasets/fastscandataset/images/test"),
        ProcessingOutput(output_name="val", source="/opt/ml/processing/input/aws_datasets/fastscandataset/raw/val", destination="s3://angkokleong-bucket/datasets/fastscandataset/images/val"),
        ProcessingOutput(output_name="train_label", source="/opt/ml/processing/input/aws_datasets/fastscandataset/labels/train", destination="s3://angkokleong-bucket/datasets/fastscandataset/labels/train"),
        ProcessingOutput(output_name="test_label", source="/opt/ml/processing/input/aws_datasets/fastscandataset/labels/test", destination="s3://angkokleong-bucket/datasets/fastscandataset/labels/test"),
        ProcessingOutput(output_name="val_label", source="/opt/ml/processing/input/aws_datasets/fastscandataset/labels/val", destination="s3://angkokleong-bucket/datasets/fastscandataset/labels/val")
    ]   
)


image_and_label_data_process_step = ProcessingStep(name="image_and_label_data_processing", step_args=processor_args)

## 4. Prepare YOLO Model Training

# 5. 

# 12. Define the Pipeline

In [25]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"FastScanPipeline"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_image_data,
        input_label_data,
        mAP50_threshold,
        mAP50to95_threshold
    ],
    steps=[image_and_label_data_process_step]
)




In [26]:
pipeline.upsert(role_arn=role)

[02/26/25 08:21:30] INFO     Uploaded None to                                                    ]8;id=614296;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=37220;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-396913742348/FastScanPipeline/code/a77d98c                   
                             d2144a0e800a4fef4836f996eadb5fbea71f264ab338cbe7cced5f748/sourcedir                   
                             .tar.gz                                                                               

                    INFO     runproc.sh uploaded to                                              ]8;id=82938;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=165428;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-396913742348/FastScanPipeline/code/562b613                   
                             02fd24a14c42a3977500317553272a0b7d2741c7404d2e02f90a46219/runproc.s                   
                             h                                                                                     

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=714121;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=900019;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[02/26/25 08:21:31] INFO     Uploaded None to                                                    ]8;id=534258;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=48455;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-396913742348/FastScanPipeline/code/a77d98c                   
                             d2144a0e800a4fef4836f996eadb5fbea71f264ab338cbe7cced5f748/sourcedir                   
                             .tar.gz                                                                               

                    INFO     runproc.sh uploaded to                                              ]8;id=896796;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=34244;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-396913742348/FastScanPipeline/code/562b613                   
                             02fd24a14c42a3977500317553272a0b7d2741c7404d2e02f90a46219/runproc.s                   
                             h                                                                                     

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=469721;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=110301;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:396913742348:pipeline/FastScanPipeline',
 'ResponseMetadata': {'RequestId': '9efca60b-0e2f-4cd6-9cf1-23b9c2c3e2de',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9efca60b-0e2f-4cd6-9cf1-23b9c2c3e2de',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Wed, 26 Feb 2025 08:21:31 GMT'},
  'RetryAttempts': 0}}

# Start the pipeline

In [27]:
execution = pipeline.start()

In [28]:
execution.wait()
execution.describe()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 execution.wait()                                                                             │
│   2 execution.describe()                                                                         │
│   3                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline.py:934 in wait               │
│                                                                                                  │
│    931 │   │   waiter = botocore.waiter.create_waiter_with_client(                               │
│    932 │   │   │   waiter_id, model, self.sagemaker_session.sagemaker_client                     │
│    933 │   │   )                                                                                 │
│ ❱  934 │   │   waiter.wait(PipelineExecutionArn=self.arn)                                        │
│    935 │                                                                                         │
│    936 │   def result(self, step_name: str):                                                     │
│    937 │   │   """Retrieves the output of the provided step if it is a ``@step`` decorated func  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/waiter.py:55 in wait                            │
│                                                                                                  │
│    52 │   # Waiter.wait method. This is needed to attach a docstring to the                      │
│    53 │   # method.                                                                              │
│    54 │   def wait(self, **kwargs):                                                              │
│ ❱  55 │   │   Waiter.wait(self, **kwargs)                                                        │
│    56 │                                                                                          │
│    57 │   wait.__doc__ = WaiterDocstring(                                                        │
│    58 │   │   waiter_name=waiter_name,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/waiter.py:374 in wait                           │
│                                                                                                  │
│   371 │   │   │   │   return                                                                     │
│   372 │   │   │   if current_state == 'failure':                                                 │
│   373 │   │   │   │   reason = f'Waiter encountered a terminal failure state: {acceptor.explan   │
│ ❱ 374 │   │   │   │   raise WaiterError(                                                         │
│   375 │   │   │   │   │   name=self.name,                                                        │
│   376 │   │   │   │   │   reason=reason,                                                         │
│   377 │   │   │   │   │   last_response=response,                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression 
"PipelineExecutionStatus" we matched expected path: "Failed"

In [ ]:
execution.list_steps()